# Spam Filter Using Naive Bayes
### Introduction

There are several classification techniques in the area of machine learning. One such technique is the Naive Bayes algorithm. The Naive Bayes is a family of classifiers, and through the use of probabilities, determines the class of an object. In this project, I endeavour to create a spam filter using the multinomial Naive Bayes algorithm. The dataset I will be using has 5572 SMS messages, and it was constructed by Tiago A. Almeida and Jose Maria Gomez Hidalgo. The dataset can be downloaded from the <a href="https://archive.ics.uci.edu/ml/datasets/sms+spam+collection" target="_blank">The UCI Machine learning Repository</a>. These messages have already been classified as spam or ham (not spam).

The goal of this project is to create an algorithm that can classify SMS messages by training it with as many already classfied messages as possible. In so doing, the algorithm will become a more effective classifier, and thus, the accuracy of our spam filter will be higher.

I begin by reading in the dataset:

In [1]:
#Import pandas library and read in the data
#The dataset does not have a header row. So, I create one.

import pandas

smsdata = pandas.read_csv(r'C:\Users\Daniel\.jupyter\SMSSpamCollection', sep = '\t', header = None, names = ['Label', 'SMS'])

### Exploring the Data

Below, I explore the data a little bit, and as can be seen, there are 5572 rows and 2 columns. You'll also find the first five rows of the dataset. Moreover, this dataset is comprised of ~87% ham messages and ~13% spam messages. 

In [2]:
#Find the number of rows and columns of the dataset
#Explore the first few rows of the dataset

print(smsdata.shape)
smsdata.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
#Find the proportion of spam and ham messages

smsdata['Label'].value_counts(normalize = True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

### Creating Training and Test Sets

In this next step, I split the original dataset to create a training set and test set. I, first, randomize the original dataset, and then assign 85% of the observations to the training set and the remaining 15% to the test set. The purpose of splitting the dataset is simply to leverage the fact that all the messages have already been classified. By performing this step, I will be able to compare the predicted labels to the actual labels, and thus, determine the accuracy of the spam filter. 

The 85/15 split is partly arbitrary, but the idea is to train the algorithm with as many messages as possible so as to have just enough observations on which to test the algorithm. 

In [4]:
#Randomize the dataset
#Split the dataset into a training set (w/ 85% of the data) and a test set (w/ 15% of the data)

rand_set = smsdata.sample(frac = 1, random_state = 1)
rand_set_index = round(len(rand_set) * 0.85)
train_set = rand_set[:rand_set_index].reset_index(drop = True)
test_set = rand_set[rand_set_index:].reset_index(drop = True)

print(train_set.shape)
print(test_set.shape)

(4736, 2)
(836, 2)


Below we see that both the training and test sets closely resemble the composition of the original dataset:

In [5]:
#Find the proportion of ham and spam messages for the training set

train_set['Label'].value_counts(normalize = True)

ham     0.865921
spam    0.134079
Name: Label, dtype: float64

In [6]:
#Find the proportion of ham and spam messages for the test set

test_set['Label'].value_counts(normalize = True)

ham     0.866029
spam    0.133971
Name: Label, dtype: float64

### Data Cleaning

The next few steps will involve cleaning the training set so that it can be used by our Naive Bayes algorithm. To understand why the next steps are necessary, it's important to understand the construction of the Naive Bayes algorithm. 

The Naive Bayes algorithm is based on Bayes' Theorem. The algorithm is 'naive' because it assumes conditional independence between elements within an object (i.e. words within a message). Hence, the formula looks like the following:

<img src="https://latex.codecogs.com/gif.latex?P(Spam|w_1,w_2,...,w_n)&space;\propto&space;P(Spam)&space;\cdot&space;{\displaystyle&space;\prod_{i=1}^{n}&space;P(w_i|Spam)}" title="P(Spam|w_1,w_2,...,w_n) \propto P(Spam) \cdot {\displaystyle \prod_{i=1}^{n} P(w_i|Spam)}" />
<img src="https://latex.codecogs.com/gif.latex?P(Ham|w_1,w_2,...,w_n)&space;\propto&space;P(Ham)&space;\cdot&space;{\displaystyle&space;\prod_{i=1}^{n}&space;P(w_i|Ham)}" title="P(Ham|w_1,w_2,...,w_n) \propto P(Ham) \cdot {\displaystyle \prod_{i=1}^{n} P(w_i|Ham)}" />

Where $\propto$ means directly proportional to and $w_i$ represents a word in the message. Notice there isn't a denominator for the above equations. This is done to optimize the algorithm. Ridding the equations of the denominators won't affect the algorithm's ability to classify new messages. Hence, the directly proportional symbol, $\propto$. 

So, in the following steps, the goal is to create a complete dataframe that includes columns for the unique words in the training set. The values for these columns will be the number of times a respective word appears in a certain message. Below is the training set before the data cleaning:


In [7]:
#Training set before data cleaning

train_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In the first step of the data cleaning process, I remove punctuations from all the messages in the training set and convert the messages to lowercase. I, then, create a vocabulary list, which is comprised of all the unique words in the training set. As seen below, there are 8,030 unique words in the training set. 

In [8]:
#Remove from the messages any punctuations
#Convert the messages to lowercase
#Training set after data cleaning

train_set['SMS'] = train_set['SMS'].str.replace('\W', ' ')
train_set['SMS'] = train_set['SMS'].str.lower()
train_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [9]:
#Create a vocabulary list, which is a set of unique words that make up the messages in the training set

train_set['SMS'] = train_set['SMS'].str.split() #The split function is to convert the string into a list of the elements

vocab = []

for sms in train_set['SMS']:
    for word in sms:
        vocab.append(word)

vocab = list(set(vocab))

In [10]:
#Find the number of words in the vocabulary

len(vocab)

8030

After having created the vocabulary list, I construct a dataframe that has the unique words of the training set comprise its columns. I, then, combine this dataframe with the training set dataframe. 

In [11]:
#Create a dataframe that has the unique words in the training set as its columns
#The values in the columns represent the number of the times a respective word appears in a certain message 

#Initialize a dictionary of unique words that have values of 0
word_counts_per_sms = {unique_word: [0] * len(train_set['SMS']) for unique_word in vocab} 

#Use enumerate function to assign an index to each sms in the training set
for index, sms in enumerate(train_set['SMS']): 
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts = pandas.DataFrame(word_counts_per_sms)
word_counts.head()

,tissco,n8,trained,someday,sec,dontcha,dnt,embarassing,rumbling,grow,...,mumhas,imagination,wildlife,embarassed,hypotheticalhuagauahahuagahyuhagga,deck,shitin,523,abuse,monos
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#Combine the training set and word counts dataframes

train_set_clean = pandas.concat([train_set, word_counts], axis = 1)
train_set_clean.head()

,Label,SMS,tissco,n8,trained,someday,sec,dontcha,dnt,embarassing,...,mumhas,imagination,wildlife,embarassed,hypotheticalhuagauahahuagahyuhagga,deck,shitin,523,abuse,monos
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Constructing the Spam Filter

Now that I've cleaned the data, I can begin creating the variables needed to construct the Naive Bayes algorithm. These variables include the probabilities of spam and ham messages, and the parameters (or probabilities of unique words given spam or ham messages). Notice that I defined a variable named alpha and stored a value of 1 in it. This is to help us calculate the parameters of the Naive Bayes algorithm, and it will be further explained below.

In [13]:
#Store spam and ham messages separate variables

spam_sms = train_set_clean[train_set_clean['Label'] == 'spam']
ham_sms = train_set_clean[train_set_clean['Label'] == 'ham']

#Store the total number of words in spam messages and ham messages in separate variables 

n_spam = spam_sms['SMS'].apply(len).sum()
n_ham = ham_sms['SMS'].apply(len).sum()

#Store number of words in vocabulary in a variable
#Define a variable called alpha and store in it a value of 1 

n_vocab = len(vocab)
alpha = 1

In [14]:
#Calculate the probability of spam and ham messages in separate variables

p_spam = len(spam_sms) / len(train_set_clean)
p_ham = len(ham_sms) / len(train_set_clean)

Below is the calculation of the parameters in our training set. A parameter represents the probability value of a particular word given spam ($ P(w_i|Spam) $) or ham ($ P(w_i|Ham) $). 

I calculate the parameters the following way:

<img src="https://latex.codecogs.com/gif.latex?P(w_i|Spam)&space;=&space;\dfrac{N_{w_i|Spam}&space;&plus;&space;\alpha}{N_{Spam}&space;&plus;&space;\alpha&space;\cdot&space;N_{Vocabulary}}" title="P(w_i|Spam) = \dfrac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}" />
<img src="https://latex.codecogs.com/gif.latex?P(w_i|Ham)&space;=&space;\dfrac{N_{w_i|Ham}&space;&plus;&space;\alpha}{N_{Ham}&space;&plus;&space;\alpha&space;\cdot&space;N_{Vocabulary}}" title="P(w_i|Ham) = \dfrac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}" />

Where $N_{w_i|Spam}$ is the number of a particular word in all spam messages, $N_{w_i|Ham}$ is the number of a particular word in all ham messages, $N_{Spam}$ is the number of words in all spam messages, and $N_{Ham}$ is the number of words in all ham messages.

The probability of a word given ham or spam is called a parameter because this probability does not change given any new message. The probability depends solely on the training set. So long as the training set does not change, our parameters do not change (hence, the term, parameter). 

In my calculation of the parameters, there's an alpha, $\alpha$, component in both the numerator and denominator, and in the denominator, there's also a term that represents the number of words in the vocabulary, $N_{vocabulary}$. This process is called 'additive smoothing'. This method is employed because if there is a word in a new message that isn't in the vocabulary, instead of attaching a probability of 0 and nullifying the product of the various probabilties in the SMS message, we add 1 to the numerator and add the product of alpha and the length of the vocabulary to the denominator, and thus, keeping the product of the probabilities intact. Additive smoothing is performed on all words in a new SMS message. 

Since there are 8,030 unique words in the training set, 16,060 parameters will be calculated.

In [15]:
#Calculate the parameters

parameters_spam = {unique_word: 0 for unique_word in vocab}
parameters_ham = {unique_word: 0 for unique_word in vocab}

for word in vocab:
    p_word_given_spam = (spam_sms[word].sum() + alpha) / (n_spam + alpha * n_vocab)
    parameters_spam[word] = p_word_given_spam
    
    p_word_given_ham = (ham_sms[word].sum() + alpha) / (n_ham + alpha * n_vocab)
    parameters_ham[word] = p_word_given_ham

Now that I've created my variables, I can construct the spam filter. The following spam filter can take in messages (either directly written messages or messages in a dataframe) and <b>prints</b> out whether the message(s) is/are ham, spam, or needs human classification.

In [16]:
#Create a spam filter that prints a classification of ham/spam/human assistance as the output

import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
   
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities. Requires human classification!') 

In [17]:
classify('WINNER!! This following is the secret code to unlock the money: C3421.')

P(Spam|message): 3.374289191685471e-29
P(Ham|message): 7.157887182915353e-32
Label: Spam


In [18]:
classify("Thanks, Dan. I'll see you tomorrow")

P(Spam|message): 2.9223605311263926e-24
P(Ham|message): 1.126173821759989e-19
Label: Ham


The following spam filter can take in messages (either directly written messages or messages in a dataframe) and <b>returns</b> whether the message(s) is/are ham, spam, or needs human classification. I apply this spam filter on the test set and store the predicted values in a newly created column called <b>'Predicted'</b>. After running the algorithm on the test set, I calculate the accuracy of the spam filter.

In [19]:
#Create a spam filter that, instead, returns ham/spam/human assistance as the output

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [20]:
#Create a new column in the test set that stores the predicted labels

test_set['Predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,Predicted
0,ham,ALSO TELL HIM I SAID HAPPY BIRTHDAY,ham
1,ham,Aiyo a bit pai seh ü noe... Scared he dun rem ...,ham
2,ham,Yes..he is really great..bhaji told kallis bes...,ham
3,ham,R u still working now?,ham
4,ham,"Thanks da thangam, i feel very very happy dear...",ham


In [21]:
#Determine the accuracy of spam filter

correct = 0
total = len(test_set)

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['Predicted']:
        correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', round((correct/total)*100, 2),'%')

Correct: 826
Incorrect: 10
Accuracy: 98.8 %


In [22]:
#Display the rows where the actual label does not match the predicted label

test_set.loc[~(test_set['Label'] == test_set['Predicted'])]

,Label,SMS,Predicted
6,ham,Nokia phone is lovly..,spam
15,ham,A Boy loved a gal. He propsd bt she didnt mind...,needs human classification
24,ham,No calls..messages..missed calls,spam
41,ham,We have sent JD for Customer Service cum Accou...,spam
226,spam,Oh my god! I've found your number again! I'm s...,ham
268,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham
463,spam,"0A$NETWORKS allow companies to bill for SMS, s...",ham
598,spam,RCT' THNQ Adrian for U text. Rgds Vatian,ham
607,spam,2/2 146tf150p,ham
675,spam,Hello. We need some posh birds and chaps to us...,ham


### Conclusion

Displayed directly above are the 10 messages for which labels were incorrectly predicted. Based on observation, it seems most of the spam messages can pass off as ham and vice versa, and thus, our spam filter did not classify these messages correctly.

Having said that, the spam filter achieved an accuracy of 98.8%, which is excellent by any standard. The accuracy of the filter would climb higher if I trained the algorithm on more SMS messages from the original dataset. By training the algorithm with more data points, the probabilities that make up the Naive Bayes would more effectively account for ham and spam messages.

In this project, I have employed the multinomiral Naive Bayes algorithm to create a spam filter. This classification technique can be used for other purposes as well such as predicting political alignment using Twitter data, the general direction of the stock market using data from daily news headlines, and etcetera.

